In [1]:
import pandas as pd
import numpy as np
import urllib.request, json 
from urllib.parse import quote

In [2]:
df = pd.read_excel('data/top100_richest_people_forbes_ua.xlsx', sheet_name='Sheet1')
df.head(5)

,Місце,"Ім'я, прізвище",Статки,Компанія/бренд,Сфера інтересів
0,1,Рінат Ахметов,"$7,6 млрд","«Метінвест», ДТЕК","Металургія, ПЕК"
1,2,Віктор Пінчук,"$2,5 млрд",«Інтерпайп»,"Металургія, нерухомість"
2,3,Костянтин Жеваго,"$2,4 млрд",Ferrexpo,Металургія
3,4,Ігор Коломойський,"$1,8 млрд",«Укрнафта»,"ЕК, інвестиції"
4,5,Геннадій Боголюбов,"$1,7 млрд",«Укрнафта»,"ПЕК, інвестиції"


In [3]:
#defining columns in English
col_names = ['Rank', 'Full name', 'Capital', 'Company/Brand', 'Branch']
#renaming columns
df = pd.DataFrame(data=df.values, columns=col_names)

#translating Branch values with Google Translate api
translation = []
for i in df['Branch']:
    with urllib.request.urlopen("https://translate.googleapis.com/translate_a/single?client=gtx&sl=uk&tl=en&dt=t&q="+quote(i)) as url:
        transl = json.load(url)[0][0][0]
    translation.append(transl)

# translating mil. and bil in Capital column
capital = []
for i in df['Capital']:
    v = i.replace('млрд', 'bil.')
    v = v.replace('млн', 'mil.')
    capital.append(v)

#replacing in the dataset
df['Capital'] = capital
df['Branch'] = translation

df.head()


,Rank,Full name,Capital,Company/Brand,Branch
0,1,Рінат Ахметов,"$7,6 bil.","«Метінвест», ДТЕК","Metallurgy, PEK"
1,2,Віктор Пінчук,"$2,5 bil.",«Інтерпайп»,"Metallurgy, real estate"
2,3,Костянтин Жеваго,"$2,4 bil.",Ferrexpo,Metallurgy
3,4,Ігор Коломойський,"$1,8 bil.",«Укрнафта»,"EC, investments"
4,5,Геннадій Боголюбов,"$1,7 bil.",«Укрнафта»,"PEK, investments"


In [4]:
# no need to translate own names - should transliterate them instead (Full name, Company columns)
import string

def translit(string):
    string = str(string)
    capital_letters = {
        u'А': u'A',
        u'Б': u'B',
        u'В': u'V',
        u'Г': u'H',
        u'Ґ': u'G',
        u'Д': u'D',
        u'Е': u'E',
        u'Ё': u'E',
        u'Ж': u'Zh',
        u'З': u'Z',
        u'И': u'Y',
        u'Й': u'Y',
        u'К': u'K',
        u'Л': u'L',
        u'М': u'M',
        u'Н': u'N',
        u'О': u'O',
        u'П': u'P',
        u'Р': u'R',
        u'С': u'S',
        u'Т': u'T',
        u'У': u'U',
        u'Ф': u'F',
        u'Х': u'H',
        u'Ц': u'Ts',
        u'Ч': u'Ch',
        u'Ш': u'Sh',
        u'Щ': u'Sch',
        u'Ъ': u'',
        u'Ы': u'Y',
        u'Ь': u'',
        u'Э': u'E',
        u'Ю': u'Yu',
        u'Я': u'Ya',
        u'Ї': u'I',
        u'Є': u'E',
        u'І': u'I'
    }

    lower_case_letters = {
        u'а': u'a',
        u'б': u'b',
        u'в': u'v',
        u'г': u'h',
        u'ґ': u'g',
        u'д': u'd',
        u'е': u'e',
        u'ё': u'e',
        u'ж': u'zh',
        u'з': u'z',
        u'и': u'y',
        u'й': u'y',
        u'к': u'k',
        u'л': u'l',
        u'м': u'm',
        u'н': u'n',
        u'о': u'o',
        u'п': u'p',
        u'р': u'r',
        u'с': u's',
        u'т': u't',
        u'у': u'u',
        u'ф': u'f',
        u'х': u'h',
        u'ц': u'ts',
        u'ч': u'ch',
        u'ш': u'sh',
        u'щ': u'sch',
        u'ъ': u'',
        u'ы': u'y',
        u'ь': u'',
        u'э': u'e',
        u'ю': u'yu',
        u'я': u'ya',
        u'ї': u'i',
        u'є': u'e',
        u'і': u'i'
    }

    translit_string = ""

    for index, char in enumerate(string):
        if char in lower_case_letters.keys():
            char = lower_case_letters[char]
        elif char in capital_letters.keys():
            char = capital_letters[char]
            if len(string) > index+1:
                if string[index+1] not in lower_case_letters.keys():
                    char = char.upper()
            else:
                char = char.upper()
        translit_string += char

    return translit_string

#applying to the dataset
rows = int(len(df))
fullname_transl = []
company_transl = []
for i in range(rows):
    fullname_transl.append(translit(df['Full name'][i]))
    company_transl.append(translit(df['Company/Brand'][i]))
df['Full name'] = fullname_transl
df['Company/Brand'] = company_transl


In [5]:
#saving as a separate csv file

df.to_csv('transformed.csv',index=False)